In [1]:
import torch
import torch.optim as optim
import torch_geometric
from torch.nn.functional import relu, sigmoid, softmax, mse_loss
from torch.nn import Linear, Module, Dropout, MSELoss, CrossEntropyLoss, BatchNorm1d
from torch.cuda.amp import autocast, GradScaler

from tqdm import tqdm

from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool, GATv2Conv

from SchNet import SchNet

import pandas as pd
import numpy as np

import os
import pickle
import gzip

os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
device = 0
device = torch.device("cuda:{}".format(device) if torch.cuda.is_available() else "cpu")

In [2]:
train = pd.read_csv('kiba/train.csv', index_col=0)
val = pd.read_csv('kiba/val.csv', index_col=0)
test = pd.read_csv('kiba/test.csv', index_col=0)

In [3]:
print('Train dim:', train.shape)
print('val dim:', val.shape)
print('test dim:', test.shape)

Train dim: (3449, 3)
val dim: (494, 3)
test dim: (973, 3)


In [4]:
with gzip.open('drug.pkl.gz', 'rb') as f:
    drug = pickle.load(f)

def get_drug_dataloader(drugs, batch_size=100):
    dataset = [drug[i] for i in drugs]
    return DataLoader(dataset, batch_size=batch_size)

def get_protein_dataloader(proteins, batch_size=100):
    dataset = [torch.load('protein_graphs/{}.pt'.format(i)) for i in proteins]
    return DataLoader(dataset, batch_size=batch_size)

In [5]:
batch_size = 5
drug_train_loader = get_drug_dataloader(train['Drug'], batch_size)
drug_val_loader = get_drug_dataloader(val['Drug'], batch_size)
drug_test_loader = get_drug_dataloader(test['Drug'], batch_size)

protein_train_loader = get_protein_dataloader(train['Target_ID'], batch_size)
protein_val_loader = get_protein_dataloader(val['Target_ID'], batch_size)
protein_test_loader = get_protein_dataloader(test['Target_ID'], batch_size)

train_y = DataLoader(torch.Tensor(train['Y'].values).float(), batch_size=batch_size)
val_y = DataLoader(torch.Tensor(val['Y'].values).float(), batch_size=batch_size)
test_y = DataLoader(torch.Tensor(test['Y'].values).float(), batch_size=batch_size)

/export/scratch/inoue019/envs/gpu1/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [6]:
class MultiHeadAttention(Module):
    def __init__(self, hidden_dim, num_heads, dropout):
        super(MultiHeadAttention, self).__init__()

        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        assert hidden_dim % num_heads == 0

        self.query_linear = Linear(hidden_dim, hidden_dim)
        self.key_linear = Linear(hidden_dim, hidden_dim)
        self.value_linear = Linear(hidden_dim, hidden_dim)

        self.output_linear = Linear(hidden_dim, hidden_dim)
        self.dropout = Dropout(dropout)

        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim // num_heads])).to(device)

    def forward(self, query, key, value, mask=None):
        batch_size = query.shape[0]

        Q = self.query_linear(query)
        K = self.key_linear(key)
        V = self.value_linear(value)

        Q = Q.view(batch_size, self.num_heads, -1, self.hidden_dim // self.num_heads)
        K = K.view(batch_size, self.num_heads, -1, self.hidden_dim // self.num_heads)
        V = V.view(batch_size, self.num_heads, -1, self.hidden_dim // self.num_heads)
    
        energy = torch.matmul(Q, K.transpose(-2, -1)) / self.scale
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)

        attention = self.dropout(softmax(energy, dim=-1)).to(torch.float16)

        weighted_matrix = torch.matmul(attention, V)

        weighted_matrix = weighted_matrix.permute(0, 2, 1, 3).contiguous()
        weighted_matrix = weighted_matrix.view(batch_size, -1, self.hidden_dim)
#         weighted_matrix = weighted_matrix.mean(dim=1)
#         weighted_matrix = torch.max(weighted_matrix, dim=1)
        weighted_matrix = weighted_matrix[:, 0, :]

        output = self.output_linear(weighted_matrix)

        return output

In [7]:
class DTIPredictor(Module):
    def __init__(self, hidden_dim=100, num_heads=1, dropout=0.2):
        super().__init__()
        self.drug_encoder = SchNet(out_dim=hidden_dim)
        self.protein_encoder = SchNet(out_dim=hidden_dim)
        self.attention = MultiHeadAttention(hidden_dim, num_heads, dropout)
        self.fc_output = Linear(hidden_dim*3, 1)

    def forward(self, drug_data, protein_data):
        x_drug = self.drug_encoder(drug_data.atom_type.int(), drug_data.pos.to(torch.float16), drug_data.batch)
        torch.cuda.empty_cache()
        x_protein = self.protein_encoder(protein_data.atom_type.int(), protein_data.pos.to(torch.float16), protein_data.batch)
        torch.cuda.empty_cache()
        attention_output = self.attention(x_drug, x_protein, x_protein)
        torch.cuda.empty_cache()

        feature = torch.cat((x_drug, attention_output, x_protein), dim=1)
        torch.cuda.empty_cache()
        prediction = self.fc_output(feature)
        torch.cuda.empty_cache()

        return (prediction).squeeze(1)

In [8]:
model = DTIPredictor().to(device).to(torch.float16)
criterion = MSELoss().to(device).to(torch.float16)
optimizer = getattr(torch.optim, "Adam")(model.parameters(), lr=0.01,)

train_losses = []
train_accs = []
val_losses = []
val_accs = []

for epoch in tqdm(range(10)):

    model.train()
    total_loss = 0
    for drug, protein, true_y in tqdm(zip(drug_train_loader, protein_train_loader, train_y)):
        drug = drug.to(device)
        protein = protein.to(device)
        true_y = true_y.to(device)
        torch.cuda.empty_cache()
        
        optimizer.zero_grad()
        torch.cuda.empty_cache()
        
        output = model(drug, protein)
        loss = criterion(output, true_y.to(torch.float16))
        torch.cuda.empty_cache()
        loss.backward()
        torch.cuda.empty_cache()
        optimizer.step()
        torch.cuda.empty_cache()
        total_loss += loss.item()
        
    average_loss = total_loss / len(drug_train_loader)
    train_losses.append(average_loss)
    
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for drug, protein, true_y in zip(drug_val_loader, protein_val_loader, val_y):
            drug = drug.to(device)
            protein = protein.to(device)
            true_y = true_y.to(device)            
            
            print(drug.shape)
            
            output = model(drug, protein)
            loss = criterion(output, true_y.to(torch.float16))
            val_loss += loss.item()
            val_losses.append(val_loss)

        average_val_loss = val_loss / len(drug_val_loader)
        val_losses.append(average_val_loss)
    print(f'Epoch: {epoch+1}, Train loss: {average_loss:.4f}, Validation Loss: {average_val_loss:.4f}')

  0%|          | 0/10 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  1.24it/s]
2it [00:01,  2.11it/s]
3it [00:01,  2.72it/s]
4it [00:01,  2.92it/s]
5it [00:01,  3.38it/s]
6it [00:02,  3.66it/s]
7it [00:02,  3.83it/s]
8it [00:02,  4.21it/s]
9it [00:02,  4.38it/s]
10it [00:02,  4.13it/s]
11it [00:03,  4.19it/s]
12it [00:03,  4.28it/s]
13it [00:03,  4.56it/s]
14it [00:03,  4.67it/s]
15it [00:04,  4.47it/s]
16it [00:04,  4.17it/s]
17it [00:04,  4.37it/s]
18it [00:04,  3.97it/s]
19it [00:05,  3.93it/s]
20it [00:05,  3.96it/s]
21it [00:05,  4.39it/s]
22it [00:05,  4.83it/s]
23it [00:05,  5.17it/s]
24it [00:06,  5.10it/s]
25it [00:06,  5.48it/s]
26it [00:06,  4.97it/s]
27it [00:06,  4.66it/s]
28it [00:06,  4.58it/s]
29it [00:07,  4.66it/s]
30it [00:07,  4.57it/s]
31it [00:07,  4.82it/s]
32it [00:07,  4.99it/s]
33it [00:07,  4.80it/s]
34it [00:08,  5.02it/s]
35it [00:08,  5.03it/s]
36it [00:08,  5.10it/s]
37it [00:08,  4.40it/s]
38it [00:09,  4.13it/s]
39it [00:09,  4.28it/s]
40it [00:09,  4

OutOfMemoryError: CUDA out of memory. Tried to allocate 370.00 MiB (GPU 0; 10.75 GiB total capacity; 9.49 GiB already allocated; 302.69 MiB free; 9.60 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
model.eval()
pred = []
with torch.no_grad():
    test_loss = 0
    for drug, protein, true_y in zip(drug_test_loader, protein_test_loader, test_y):
        drug = drug.to(device)
        protein = protein.to(device)
        true_y = true_y.to(device)            

        output = model(drug, protein)
        loss = criterion(output, true_y)
        test_loss += loss.item()
        pred.append(output.cpu().detach().numpy()) 
        
pred = np.concatenate(pred)

In [ ]:
mse_loss(torch.Tensor(pred), torch.Tensor(np.array(test['Y'])))